# Notebook #6: Predicciones

In [61]:
import pandas as pd
import pickle
import numpy as np

In [62]:
# cargamos los transformadores y el modelo entrenado
with open('../../modelos/onehot_encoder.pkl', 'rb') as f:
    onehot_encoder = pickle.load(f)

with open('../../modelos/target_encoder.pkl', 'rb') as f:
    target_encoder = pickle.load(f)

with open('../../modelos/standard_scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

with open('../../modelos/rf_regressor.pkl', 'rb') as f:
    model = pickle.load(f)

In [63]:
#'price', 'propertyType', 'size', 'exterior', 'rooms', 'bathrooms',
#       'municipality', 'distance', 'floor', 'hasLift'

In [64]:
nueva_casa = pd.DataFrame({
    'propertyType': ["Flat"], 
    'size': [60],
    'exterior': ["True"],
    'rooms': [2],
    'bathrooms': [1],
    'municipality':["Madrid"],
    'distance':[2000],
    'floor':[2],
    'hasLift':["True"]
})

df_new = pd.DataFrame(nueva_casa)
df_new

,propertyType,size,exterior,rooms,bathrooms,municipality,distance,floor,hasLift
0,Flat,60,True,2,1,Madrid,2000,2,True


In [76]:
df_pred = df_new.copy()
col_num=["size", "distance"]
df_pred[col_num] = scaler.transform(df_pred[col_num])
df_pred['hasLift'] = df_pred['hasLift'].map({
    "True": 1, 
    "False": 0, 
    'Sin información': -1
})

df_pred['exterior'] = df_pred['exterior'].map({
    "True": 1, 
    "False": 0, 
    'Sin información': -1
})

df_pred['floor'] = df_pred['floor'].map({
    'st': -2, 
    'ss': -1,
    'bj': 0,
    'en': 0.5, 
    'Sin información': -3
}).fillna(df_pred['floor']) 

df_pred["municipality"] = target_encoder.transform(df_pred["municipality"])

# Transformar columna
encoded_array = onehot_encoder.transform(df_pred[["propertyType"]])

# Crear un DF con las nuevas columnas
encoded_df = pd.DataFrame(
    encoded_array,
    columns=onehot_encoder.get_feature_names_out(["propertyType"]),
    index=df_pred.index
)

# Concatenar con el DF original
df_pred = pd.concat([df_pred.reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)

# Drop de la columna original que ya no necesitamos
df_pred.drop("propertyType", axis=1, inplace=True)

df_pred.rename(columns= {"size":"size_standard", "distance":"distance_standard"}, inplace=True)

df_pred = df_pred[['exterior', 'rooms', 'bathrooms', 'floor', 'hasLift',
       'propertyType_chalet', 'propertyType_countryHouse',
       'propertyType_duplex', 'propertyType_flat', 'propertyType_penthouse',
       'propertyType_studio', 'municipality', 'size_standard',
       'distance_standard']]

model.predict(df_pred)[0]

np.float64(716.8056014628107)